## Day 17

https://adventofcode.com/2020/day/17

In [1]:
import collections
import itertools
import typing

In [2]:
import aocd

In [3]:
ACTIVE = '#'
INACTIVE = '.'

In [4]:
data = aocd.get_data(day=17, year=2020).splitlines()
len(data)

8

In [5]:
test_data = [
    '.#.',
    '..#',
    '###',
]

In [6]:
class Point(typing.NamedTuple):
    x: int
    y: int
    z: int
        
    def __repr__(self):
        return f'({self.x}, {self.y}, {self.z})'
    
    def get_all_neighbors(self):
        result = [
            Point(self.x + i, self.y + j, self.z + k)
            for i, j, k in itertools.product([-1, 0, 1], repeat=3)
            if (i, j, k) != (0, 0, 0)
        ]
        assert len(result) == 26
        return set(result)     

In [7]:
class Grid(collections.abc.Set):
    def __init__(self, active_cells):
        self._active = set(active_cells)
        self.x_min = min(c.x for c in self._active)
        self.x_max = max(c.x for c in self._active)
        self.y_min = min(c.y for c in self._active)
        self.y_max = max(c.y for c in self._active)
        self.z_min = min(c.z for c in self._active)
        self.z_max = max(c.z for c in self._active)
    
    @classmethod
    def from_lines(cls, lines):
        values = {
            Point(x=i, y=j, z=0): value
            for j, row in enumerate(lines)
            for i, value in enumerate(row)
        }
        active_cells = [
            cell for cell in values
            if values[cell] == ACTIVE
        ]
        return cls(active_cells)

    def __contains__(self, cell: Point):
        return cell in self._active

    def __len__(self):
        return len(self._active)
    
    def __iter__(self):
        return iter(self._active)
    
    def is_active(self, cell: Point) -> bool:
        return cell in self
    
    def is_inactive(self, cell: Point) -> bool:
        return not cell in self
    
    @property
    def active_cells(self):
        return set(self._active)

    @property
    def potential_next_cells(self):
        return set([
            neighbor for cell in self
            for neighbor in cell.get_all_neighbors()
        ])

    @property
    def domain(self):
        return (
            (self.x_min, self.x_max),
            (self.y_min, self.y_max),
            (self.z_min, self.z_max),
        )
        
    def __repr__(self):
        x_range, y_range, z_range = self.domain
        return (
            f'<{self.__class__.__name__}('
            f'x_range={x_range}'
            f', y_range={y_range}'
            f', z_range={z_range})>'
        )
    
    def all_cells_for_layer(self, layer: int):
        return [
            [
                Point(x, y, layer)
                for x in range(self.x_min, self.x_max + 1)
            ]
            for y in range(self.y_min, self.y_max + 1)
        ]

    def print_layer(self, layer: int):
        lines = [
            ''.join(
                    ACTIVE if self.is_active(cell) else INACTIVE
                    for cell in row
            )
            for row in self.all_cells_for_layer(layer)
        ]
        print(*lines, sep='\n')
    
    def count_active_neighbors(self, cell) -> int:
        neighbors = set([
            n for n in cell.get_all_neighbors()
            if n in self
        ])
        return len([n for n in neighbors if self.is_active(n)])

In [8]:
grid = Grid.from_lines(data)
grid

<Grid(x_range=(0, 7), y_range=(0, 7), z_range=(0, 0))>

In [9]:
grid.print_layer(0)

#...#.#.
..#.#.##
..#..#..
.....###
...#.#.#
#.#.##..
#####...
.#.#.##.


### Solution to Part 1

In [10]:
def compute_new_cell(cell, *, grid) -> str:
    n = grid.count_active_neighbors(cell)
    if grid.is_active(cell):
        return ACTIVE if n in (2, 3) else INACTIVE
    elif grid.is_inactive(cell):
        return ACTIVE if n == 3 else INACTIVE
    else:
        raise RuntimeError('cannot happen')

In [11]:
def run_cycle(grid, *, grid_type=Grid):
    return grid_type([
        cell for cell in grid.potential_next_cells
        if compute_new_cell(cell, grid=grid) == ACTIVE
    ])

In [12]:
def simulate(grid, *, n: int = 6, grid_type=Grid):
    old_grid = grid_type(grid)
    for _ in range(n):
        new_grid = run_cycle(old_grid, grid_type=grid_type)
        old_grid = grid_type(new_grid)
    return new_grid

In [13]:
final_grid = simulate(grid)
len(final_grid.active_cells)

401

### Solution to Part 2

In [14]:
class Point4D(typing.NamedTuple):
    x: int
    y: int
    z: int
    w: int
        
    def __repr__(self):
        return f'({self.x}, {self.y}, {self.z}, {self.w})'
    
    def get_all_neighbors(self):
        result = [
            Point4D(self.x + i, self.y + j, self.z + k, self.w + m)
            for i, j, k, m in itertools.product([-1, 0, 1], repeat=4)
            if (i, j, k, m) != (0, 0, 0, 0)
        ]
        assert len(result) == 80
        return set(result)     

In [15]:
class Grid4D(collections.abc.Set):
    def __init__(self, active_cells):
        self._active = set(active_cells)
        self.x_min = min(c.x for c in self._active)
        self.x_max = max(c.x for c in self._active)
        self.y_min = min(c.y for c in self._active)
        self.y_max = max(c.y for c in self._active)
        self.z_min = min(c.z for c in self._active)
        self.z_max = max(c.z for c in self._active)
        self.w_min = min(c.w for c in self._active)
        self.w_max = max(c.w for c in self._active)

    
    @classmethod
    def from_lines(cls, lines):
        values = {
            Point4D(x=i, y=j, z=0, w=0): value
            for j, row in enumerate(lines)
            for i, value in enumerate(row)
        }
        active_cells = [
            cell for cell in values
            if values[cell] == ACTIVE
        ]
        return cls(active_cells)

    def __contains__(self, cell: Point):
        return cell in self._active

    def __len__(self):
        return len(self._active)
    
    def __iter__(self):
        return iter(self._active)
    
    def is_active(self, cell: Point) -> bool:
        return cell in self
    
    def is_inactive(self, cell: Point) -> bool:
        return not cell in self
    
    @property
    def active_cells(self):
        return set(self._active)

    @property
    def potential_next_cells(self):
        return set([
            neighbor for cell in self
            for neighbor in cell.get_all_neighbors()
        ])

    @property
    def domain(self):
        return (
            (self.x_min, self.x_max),
            (self.y_min, self.y_max),
            (self.z_min, self.z_max),
            (self.w_min, self.w_max),
        )
        
    def __repr__(self):
        x_range, y_range, z_range, w_range = self.domain
        return (
            f'<{self.__class__.__name__}('
            f'x_range={x_range}'
            f', y_range={y_range}'
            f', z_range={z_range}'
            f', w_range={w_range})>'
        )
    
    def all_cells_for_layer(self, *, z: int, w: int):
        return [
            [
                Point4D(x, y, z, w)
                for x in range(self.x_min, self.x_max + 1)
            ]
            for y in range(self.y_min, self.y_max + 1)
        ]

    def print_layer(self, *, z: int, w: int):
        lines = [
            ''.join(
                    ACTIVE if self.is_active(cell) else INACTIVE
                    for cell in row
            )
            for row in self.all_cells_for_layer(z=z, w=w)
        ]
        print(*lines, sep='\n')
    
    def count_active_neighbors(self, cell) -> int:
        neighbors = set([
            n for n in cell.get_all_neighbors()
            if n in self
        ])
        return len([n for n in neighbors if self.is_active(n)])

In [16]:
grid4D = Grid4D.from_lines(data)
grid4D

<Grid4D(x_range=(0, 7), y_range=(0, 7), z_range=(0, 0), w_range=(0, 0))>

In [17]:
final_grid = simulate(grid4D, grid_type=Grid4D)
len(final_grid.active_cells)

2224